# Markdown Meeting Notes, Google Doc (Google Docs API)

This Colab converts the provided markdown meeting notes into a formatted Google Doc.


In [ ]:

!pip -q install google-api-python-client google-auth-httplib2 google-auth-oauthlib markdown-it-py reportlab

In [ ]:

from google.colab import auth
auth.authenticate_user()
print("Authenticated.")

In [ ]:

MARKDOWN_NOTES = r'''

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
'''
print(MARKDOWN_NOTES.splitlines()[0])

In [ ]:
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict

@dataclass
class Paragraph:
    kind: str  
    text: str
    level: int = 0  
    mentions: Tuple[Tuple[int,int], ...] = ()  

MENTION_RE = re.compile(r'@\w+')

def _mention_spans(s: str) -> Tuple[Tuple[int,int], ...]:
    return tuple((m.start(), m.end()) for m in MENTION_RE.finditer(s))

def parse_markdown(md: str) -> List[Paragraph]:
    """Minimal markdown parser for headings, bullets, checkboxes, hr, and footer."""
    lines = md.replace('\r\n','\n').replace('\r','\n').split('\n')

    paragraphs: List[Paragraph] = []
    in_footer = False
    for raw in lines:
        line = raw.rstrip()

        if not line.strip():
            
            paragraphs.append(Paragraph(kind='text', text=''))
            continue

        if line.strip() == '---':
            paragraphs.append(Paragraph(kind='hr', text=''))
            in_footer = True
            continue

        if line.startswith('# '):
            
            title = line[2:].strip()
            
            main, sep, rest = title.partition(' - ')
            paragraphs.append(Paragraph(kind='h1', text=main.strip()))
            if rest.strip():
                paragraphs.append(Paragraph(kind='text', text=rest.strip()))
            continue

        if line.startswith('## '):
            paragraphs.append(Paragraph(kind='h2', text=line[3:].strip()))
            continue

        if line.startswith('### '):
            paragraphs.append(Paragraph(kind='h3', text=line[4:].strip()))
            continue

        
        m_cb = re.match(r'^(\s*)-\s*\[ \]\s+(.*)$', line)
        if m_cb:
            indent = len(m_cb.group(1))
            level = indent // 2
            txt = m_cb.group(2).strip()
            paragraphs.append(Paragraph(kind='checkbox', text=txt, level=level, mentions=_mention_spans(txt)))
            continue

        
        m_b = re.match(r'^(\s*)([-*])\s+(.*)$', line)
        if m_b:
            indent = len(m_b.group(1))
            level = indent // 2
            txt = m_b.group(3).strip()
            kind = 'footer' if in_footer else 'bullet'
            paragraphs.append(Paragraph(kind=kind, text=txt, level=level, mentions=_mention_spans(txt)))
            continue

        
        kind = 'footer' if in_footer else 'text'
        paragraphs.append(Paragraph(kind=kind, text=line.strip(), mentions=_mention_spans(line.strip())))

    
    return paragraphs

parsed = parse_markdown(MARKDOWN_NOTES)
print("Parsed paragraphs:", len(parsed))
print("First 8:", parsed[:8])

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.auth
from typing import Any

DOCS_SCOPES = ["https://www.googleapis.com/auth/documents"]

def create_formatted_doc(paragraphs: List[Paragraph], doc_title: str = "Product Team Sync") -> str:
    
    try:
        creds, _ = google.auth.default(scopes=DOCS_SCOPES)
        service = build("docs", "v1", credentials=creds)
    except Exception as e:
        raise RuntimeError(f"Auth/build failed: {e}") from e

    
    try:
        doc = service.documents().create(body={"title": doc_title}).execute()
        doc_id = doc["documentId"]
    except HttpError as e:
        raise RuntimeError(f"Failed to create doc (is Docs API enabled?): {e}") from e

    
    idx = 1

    
    requests: List[Dict[str, Any]] = []
    para_ranges: List[Dict[str, Any]] = []   
    mention_ranges: List[Tuple[int,int]] = []  
    footer_ranges: List[Tuple[int,int]] = []   

    def add_text(text: str):
        nonlocal idx
        
        requests.append({
            "insertText": {
                "location": {"index": idx},
                "text": text + "\n"
            }
        })
        start = idx
        end = idx + len(text) + 1  
        idx = end
        return start, end

    for p in paragraphs:
        if p.kind == "hr":
            
            add_text("")
            requests.append({"insertHorizontalRule": {"location": {"index": idx}}})
            idx += 1  
            add_text("")
            continue

        start, end = add_text(p.text)
        para_ranges.append({"kind": p.kind, "level": p.level, "start": start, "end": end, "text": p.text})

        
        for (ms, me) in p.mentions:
            mention_ranges.append((start + ms, start + me))

        if p.kind == "footer":
            footer_ranges.append((start, end - 1))  

    
    style_reqs: List[Dict[str, Any]] = []

    def update_paragraph_named_style(start: int, end: int, style: str):
        style_reqs.append({
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {"namedStyleType": style},
                "fields": "namedStyleType"
            }
        })

    def set_indent(start: int, end: int, level: int):
        
        indent_start = 36 * level
        style_reqs.append({
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {
                    "indentStart": {"magnitude": indent_start, "unit": "PT"},
                },
                "fields": "indentStart"
            }
        })

    
    for pr in para_ranges:
        if pr["kind"] == "h1":
            update_paragraph_named_style(pr["start"], pr["end"], "HEADING_1")
        elif pr["kind"] == "h2":
            update_paragraph_named_style(pr["start"], pr["end"], "HEADING_2")
        elif pr["kind"] == "h3":
            update_paragraph_named_style(pr["start"], pr["end"], "HEADING_3")

    
    for pr in para_ranges:
        if pr["kind"] in ("bullet", "footer", "checkbox"):
            
            pass

    
    for pr in para_ranges:
        if pr["kind"] in ("bullet",):
            set_indent(pr["start"], pr["end"], pr["level"])
            style_reqs.append({
                "createParagraphBullets": {
                    "range": {"startIndex": pr["start"], "endIndex": pr["end"]},
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })

        if pr["kind"] == "checkbox":
            set_indent(pr["start"], pr["end"], pr["level"])
            style_reqs.append({
                "createParagraphBullets": {
                    "range": {"startIndex": pr["start"], "endIndex": pr["end"]},
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })

    
    for (s, e) in mention_ranges:
        style_reqs.append({
            "updateTextStyle": {
                "range": {"startIndex": s, "endIndex": e},
                "textStyle": {
                    "bold": True,
                    "foregroundColor": {"color": {"rgbColor": {"red": 0.10, "green": 0.35, "blue": 0.85}}}
                },
                "fields": "bold,foregroundColor"
            }
        })

    
    for (s, e) in footer_ranges:
        style_reqs.append({
            "updateTextStyle": {
                "range": {"startIndex": s, "endIndex": e},
                "textStyle": {
                    "italic": True,
                    "foregroundColor": {"color": {"rgbColor": {"red": 0.45, "green": 0.45, "blue": 0.45}}},
                    "fontSize": {"magnitude": 9, "unit": "PT"},
                },
                "fields": "italic,foregroundColor,fontSize"
            }
        })

    
    try:
        service.documents().batchUpdate(
            documentId=doc_id,
            body={"requests": requests + style_reqs}
        ).execute()
    except HttpError as e:
        raise RuntimeError(f"Docs batchUpdate failed: {e}") from e

    return f"https://docs.google.com/document/d/{doc_id}/edit"

doc_url = create_formatted_doc(parsed, doc_title="Product Team Sync")
print("Created Google Doc:", doc_url)

In [ ]:

from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from datetime import datetime

def generate_submission_pdf(
    out_path: str,
    doc_url: str,
    github_repo_url: str = "",
    colab_notebook_url: str = "",
) -> str:
    c = canvas.Canvas(out_path, pagesize=LETTER)
    width, height = LETTER

    x = 0.8*inch
    y = height - 0.9*inch

    c.setFont("Helvetica-Bold", 16)
    c.drawString(x, y, "Submission - Markdown to Google Doc")
    y -= 0.35*inch

    c.setFont("Helvetica", 10)
    c.drawString(x, y, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    y -= 0.35*inch

    c.setFont("Helvetica-Bold", 11)
    c.drawString(x, y, "Links")
    y -= 0.2*inch

    def draw_link(label: str, url: str):
        nonlocal y
        c.setFont("Helvetica", 10)
        c.drawString(x, y, f"{label}:")
        c.setFillColorRGB(0.10, 0.35, 0.85)
        c.drawString(x + 110, y, url if url else "(add link)")
        c.setFillColorRGB(0, 0, 0)
        if url:
            c.linkURL(url, (x + 110, y-2, x + 110 + 380, y + 10), relative=0)
        y -= 0.22*inch

    draw_link("Google Doc", doc_url)
    draw_link("GitHub Repo", github_repo_url)
    draw_link("Colab Notebook", colab_notebook_url)

    y -= 0.25*inch
    c.setFont("Helvetica-Bold", 11)
    c.drawString(x, y, "Notes")
    y -= 0.18*inch

    c.setFont("Helvetica", 10)
    notes = [
        "This PDF includes links to the generated Google Doc and the code artifacts.",
        "If a link field is blank, paste the URL and re-run this cell."
    ]
    for n in notes:
        c.drawString(x, y, "- " + n)
        y -= 0.18*inch

    c.showPage()
    c.save()
    return out_path

pdf_path = "/content/submission_links.pdf"
generate_submission_pdf(pdf_path, doc_url=doc_url)
print("Wrote:", pdf_path)